# 対称性関数の計算

構造とエネルギーの関係を学ばせるにあたって、まず構造をニューラルネットワーク的に扱いやすいデータに変換する必要があります。
その方法の一つが対称性関数です。詳細は参考文献を見てもらうとして、ざっくりした説明としては、動径分布関数や角度の分布関数を工夫したようなものと考えておくと良いでしょう。

対称性関数はそこまでややこしい式ではないので、自力実装も可能ですが、近傍原子の判定などを効率よく書くのが難しいです。昔、私がpythonで自力で書いたものはとんでもなく遅かったので、最適化されているライブラリを使います。それが[DScribe](https://singroup.github.io/dscribe/latest/index.html)です。（最適化されているとはいえ、周期境界条件を入れると結構時間がかかるので、計算結果は保存しておくほうが効率が良いです）

In [1]:
from dscribe.descriptors import ACSF

nacsf=11
# Setting up the ACSF descriptor
acsf = ACSF(
    species=["Si"],
    rcut=7.0,
    g2_params=[[0.025,0.0], [0.05,0.0], [0.1,0.0],[0.2,0.0], [0.4,0.0],[0.8,0.0]],
    g4_params=[[0.1, 1.0, 1], [0.1, 4.0, 1], [0.1, 1.0, -1], [0.1, 4.0, -1]],
    periodic=True
)

用意してある計算結果を読み込んで処理します。xyzという第一原理計算やMDの業界でよく使う形式で、座標やエネルギーの値をまとめてあります。これをASEというパッケージの機能を使って処理しています。

In [2]:
from ase.io import iread
xyzfile='sample50.xyz'

import numpy as np

atoms=iread(xyzfile,index=':', format='extxyz')
nsample=1250
desc=np.zeros((nsample,64,nacsf))
label=np.zeros(nsample)


In [3]:
for ia, a in enumerate(atoms):


    all=np.arange(len(a.positions))

    acsf_Si = acsf.create(a)
    desc[ia,:,:]=acsf_Si
    label[ia]=a.get_total_energy()/64.0

ニューラルネットワークのフレームワークであるpytorchが扱いやすいように単精度に変換します。（機械学習フレームワークはだいたい単精度を使うことが多いです）

In [4]:
desc=np.float32(desc)
label=np.float32(label)

使い回すために、numpyの機能でファイルに保存しておきます

In [5]:
np.save('desc',desc)
np.save('label',label)